Virkar en er mjög hægt, af einhverjum ástæðum virkar skrapið best þegar tími milli hvers leiks er mikill(time sleep) svo þetta er frekar hægt.
Todo er að sækja líka match report, tokeniza það og hafa sem feature í gagnasettinu.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/126.0.6478.126/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [6]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller

def get_match_urls(season_url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chromedriver_autoinstaller.install()

    driver = webdriver.Chrome(options=chrome_options)
#skröpum
    try:
        driver.get(season_url)
        try:
            accept_cookies_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
            accept_cookies_button.click()
            print("Accepted cookies.")
        except:
            print("No accept cookies banner detected or button not clickable.")


        match_urls = []

        # Scroll down to trigger dynamic loading of additional matches
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(3)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find all div elements with data-href attribute containing match URLs
        match_elements = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-href]')))

        # Extract URLs from data-href attribute of each match element
        for element in match_elements:
            match_url = element.get_attribute('data-href')
            if match_url:
                # Ensure the URL starts with "https:"
                full_url = f"https:{match_url}" if match_url.startswith("//") else match_url
                match_urls.append(full_url)

        print(f"Total match URLs collected: {len(match_urls)}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        match_urls = []

    finally:
        # Close the WebDriver
        driver.quit()

    return match_urls

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import time

def scrape_premier_league_matches(match_urls):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chromedriver_autoinstaller.install()

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://premierleague.com")

    #samþykkja cookies
    try:
        accept_cookies_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Accepted cookies.")
    except:
        print("No accept cookies banner detected or button not clickable.")

    #geyma leiki og url sem skila villu af einhverjum ástæðum
    dataframes = []
    failed_urls = []

    def scrape_match(url):
        try:
            if not url.startswith("http://") and not url.startswith("https://"):
                url = "https://" + url.lstrip('/')

            driver.get(url)

            time.sleep(3)

            match_date_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='mainContent']/div/section[2]/div[2]/section/div[1]/div[1]/div[1]")))
            match_date = match_date_element.text.strip()
            print(f"Match Date: {match_date}")

            home_team_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='mainContent']/div/section[2]/div[2]/section/div[1]/div[2]/div[1]/div[1]/div[1]/a[2]/span[1]")))
            home_team = home_team_element.text.strip()
            print(f"Home Team: {home_team}")

            away_team_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='mainContent']/div/section[2]/div[2]/section/div[1]/div[2]/div[1]/div[3]/div[1]/a[2]/span[1]")))
            away_team = away_team_element.text.strip()
            print(f"Away Team: {away_team}")

            stats_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@role='tab' and @data-tab-index='2']")))
            driver.execute_script("arguments[0].click();", stats_button)
            print("Clicked on Stats button.")

            stats_table = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, "//div[@data-ui-tab='Match Stats']/table")))

            table_html = stats_table.get_attribute('outerHTML')

            #breytum töflu í df
            df = pd.read_html(table_html)[0]

            if df.empty:
                print(f"Scraped DataFrame is empty for URL: {url}")
                return None

            #laga df til
            df.columns = [home_team, 'Stat', away_team]
            df = df.dropna(how='all')
            df = df.reset_index(drop=True)
            stats_dict = {
                'Date': match_date,
                'Home_Team': home_team,
                'Away_Team': away_team
            }

            for index_row, row in df.iterrows():
                stat = row['Stat'].replace(' ', '_')
                stats_dict[f'Home_{stat}'] = row[home_team]
                stats_dict[f'Away_{stat}'] = row[away_team]

            match_df = pd.DataFrame([stats_dict])

            match_df['Date'] = pd.to_datetime(match_df['Date'], format='%a %d %b %Y').dt.strftime('%d/%m/%y')

            match_df['Home_team_correct'] = home_team
            match_df['Away_team_correct'] = away_team

            return match_df

        except Exception as e:
            print(f"Error occurred for match URL {url}: {str(e)}")
            return None

    try:
        for index, url in enumerate(match_urls, start=1):
            retries = 3
            success = False
            while retries > 0 and not success:
                match_df = scrape_match(url)
                if match_df is not None:
                    dataframes.append(match_df)

                    globals()[f'final_df{index}'] = match_df

                    #bíðum milli leikja
                    time.sleep(2)
                    success = True
                else:
                    retries -= 1
                    print(f"Retrying for URL {url}... {retries} retries left.")
            if not success:
                failed_urls.append(url)

    except Exception as e:
        print(f"Error occurred: {str(e)}")

    finally:
        driver.quit()

    print("Scraping complete.")

    print(f"Failed URLs: {failed_urls}")

    #df með heppnuðum og misheppnuðum urls
    final_combined_df = pd.concat(dataframes, ignore_index=True)
    return final_combined_df, failed_urls


In [ ]:
#2324
season_url = "https://www.premierleague.com/results?co=1&se=578&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season2324.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season2324.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#2223
season_url = "https://www.premierleague.com/results?co=1&se=489&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season2223.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season2223.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#2122
season_url = "https://www.premierleague.com/results?co=1&se=418&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season2122.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season2122.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#2021
season_url = "https://www.premierleague.com/results?co=1&se=363&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season2021.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season2021.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#1920
season_url = "https://www.premierleague.com/results?co=1&se=274&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season1920.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season1920.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#1819
season_url = "https://www.premierleague.com/results?co=1&se=210&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season1819.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season1819.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#1718
season_url = "https://www.premierleague.com/results?co=1&se=79&cl=-1"
match_urls = get_match_urls(season_url)
print(match_urls)

final_combined_df, failed_urls = scrape_premier_league_matches(match_urls)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_season1718.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_season1718.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
#Vistum nú öll saman og skröpum öll failed urls, sorterum svo eftir dagsetningu og lögum NaN
import pickle

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season1718.pickle','rb')
Eploffical_season1718 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season1819.pickle','rb')
Eploffical_season1819 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season1920.pickle','rb')
Eploffical_season1920 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season2021.pickle','rb')
Eploffical_season2021 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season2122.pickle','rb')
Eploffical_season2122 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season2223.pickle','rb')
Eploffical_season2223 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_season2324.pickle','rb')
Eploffical_season2324 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season1718.pickle','rb')
failed_urls_season1718 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season1819.pickle','rb')
failed_urls_season1819 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season1920.pickle','rb')
failed_urls_season1920 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season2021.pickle','rb')
failed_urls_season2021 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season2122.pickle','rb')
failed_urls_season2122 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season2223.pickle','rb')
failed_urls_season2223 = pickle.load(pick_read)

pick_read = open('drive/My Drive/Salka PL ML/data/failed_urls_season2324.pickle','rb')
failed_urls_season2324 = pickle.load(pick_read)
pick_read.close()

In [ ]:
#concat seasons
import pandas as pd

Epl_official = pd.concat([Eploffical_season1718, Eploffical_season1819, Eploffical_season1920, Eploffical_season2021,
                         Eploffical_season2122, Eploffical_season2223, Eploffical_season2324,], ignore_index=True)


In [ ]:
#concat failed urls and scrape
failed_urls_all = failed_urls_season1718 + failed_urls_season1819 + failed_urls_season1920 + failed_urls_season2021 + failed_urls_season2122 + failed_urls_season2223 + failed_urls_season2324


In [ ]:
final_combined_df, failed_urls = scrape_premier_league_matches(failed_urls_all)
print(final_combined_df)
print(f"Failed URLs: {failed_urls}")

In [ ]:
import pickle

pick_insert = open('drive/My Drive/Salka PL ML/data/Eploffical_failed.pickle','wb')
pickle.dump(final_combined_df, pick_insert)
pick_insert.close()

pick_insert = open('drive/My Drive/Salka PL ML/data/failed_urls_dump.pickle','wb')
pickle.dump(failed_urls, pick_insert)
pick_insert.close()

In [ ]:
pick_read = open('drive/My Drive/Salka PL ML/data/Eploffical_failed.pickle','rb')
Eploffical_failed = pickle.load(pick_read)
pick_read.close()

In [ ]:
Eploffical_failed #birtast rétt

,Date,Home_Team,Away_Team,Home_Possession_%,Away_Possession_%,Home_Shots_on_target,Away_Shots_on_target,Home_Shots,Away_Shots,Home_Touches,...,Home_Offsides,Away_Offsides,Home_Yellow_cards,Away_Yellow_cards,Home_Fouls_conceded,Away_Fouls_conceded,Home_team_correct,Away_team_correct,Home_Red_cards,Away_Red_cards
0,13/05/18,Huddersfield,Arsenal,45.0,55.0,3.0,4.0,19.0,9.0,603.0,...,0.0,2.0,1.0,0.0,11.0,7.0,Huddersfield,Arsenal,NaN,NaN
1,28/11/17,West Brom,Newcastle,47.9,52.1,4.0,4.0,8.0,14.0,610.0,...,0.0,2.0,NaN,NaN,13.0,12.0,West Brom,Newcastle,NaN,NaN
2,26/07/20,Newcastle,Liverpool,25.3,74.7,2.0,6.0,3.0,14.0,428.0,...,3.0,1.0,1.0,0.0,11.0,5.0,Newcastle,Liverpool,NaN,NaN
3,14/07/20,Chelsea,Norwich,67.7,32.3,6.0,0.0,22.0,2.0,950.0,...,0.0,1.0,2.0,2.0,10.0,9.0,Chelsea,Norwich,NaN,NaN
4,12/07/20,Aston Villa,Crystal Palace,44.9,55.1,9.0,5.0,12.0,9.0,494.0,...,1.0,0.0,2.0,4.0,21.0,21.0,Aston Villa,Crystal Palace,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,27/12/23,Chelsea,Crystal Palace,63.8,36.2,4.0,5.0,9.0,13.0,843.0,...,3.0,6.0,3.0,2.0,13.0,16.0,Chelsea,Crystal Palace,NaN,NaN
100,28/10/23,Wolves,Newcastle,43.2,56.8,6.0,5.0,11.0,13.0,598.0,...,2.0,0.0,1.0,4.0,13.0,15.0,Wolves,Newcastle,NaN,NaN
101,08/10/23,Arsenal,Man City,49.0,51.0,2.0,1.0,12.0,4.0,667.0,...,2.0,0.0,2.0,3.0,8.0,7.0,Arsenal,Man City,NaN,NaN
102,07/10/23,Luton,Spurs,33.1,66.9,2.0,4.0,12.0,15.0,475.0,...,4.0,4.0,2.0,1.0,16.0,7.0,Luton,Spurs,0.0,1.0


In [ ]:
Epl_official_full = pd.concat([Epl_official, Eploffical_failed], ignore_index=True)

In [ ]:
Epl_official_full['Date'] = pd.to_datetime(Epl_official_full['Date'], format='%d/%m/%y')
Epl_official_full['Date'] = Epl_official_full['Date'].dt.strftime('%Y/%m/%d')

In [ ]:
Epl_official_ready = Epl_official_full.sort_values(by='Date')

In [ ]:
Epl_official_ready = Epl_official_ready.drop(['Home_team_correct',	'Away_team_correct',	'Home_Yellow_cards',
                                              'Away_Yellow_cards',	'Home_Red_cards',	'Away_Red_cards'], axis=1)

In [ ]:
#ath nan ef rangstæður eru 0. lögum
Epl_official_ready = Epl_official_ready.fillna(0)
Epl_official_ready.isna().sum()

Date                    0
Home_Team               0
Away_Team               0
Home_Possession_%       0
Away_Possession_%       0
Home_Shots_on_target    0
Away_Shots_on_target    0
Home_Shots              0
Away_Shots              0
Home_Touches            0
Away_Touches            0
Home_Passes             0
Away_Passes             0
Home_Tackles            0
Away_Tackles            0
Home_Clearances         0
Away_Clearances         0
Home_Corners            0
Away_Corners            0
Home_Offsides           0
Away_Offsides           0
Home_Fouls_conceded     0
Away_Fouls_conceded     0
dtype: int64

In [ ]:
#vistum sem pickle, tilbúið til hreinsunar og sameiningar.
import pickle

pick_insert = #some path
pickle.dump(Epl_official_ready, pick_insert)
pick_insert.close()